In [ ]:
import os
import pandas as pd
from energyemissionsregio.config import DATA_PATH
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
import seaborn as sns

### input variables

In [ ]:
lau_df = None

direct_vars = ["population", 
             "area", "railway_network",
       "road_network",
       "number_of_buildings",
       "number_of_iron_and_steel_industries",
      "number_of_cement_industries",
      "number_of_refineries",
      "number_of_paper_and_printing_industries",
      "number_of_chemical_industries",
      "number_of_glass_industries",
      "number_of_non_ferrous_metals_industries",
      "number_of_non_metallic_minerals_industries",
       'average_air_pollution_due_to_pm25',
       'average_air_pollution_due_to_no2',
       'average_air_pollution_due_to_o3',
       'average_air_pollution_due_to_pm10',
      ]


for var_name in direct_vars:
    _df = pd.read_csv(
            os.path.join(DATA_PATH, f"{var_name}.csv")
        )
    _df = _df[_df["region_code"].str.startswith(("ES", "DE"))][["region_code", "value"]].copy()
    _df.rename(columns={"value": var_name}, inplace=True)

    if lau_df is not None:
        lau_df = pd.merge(lau_df, _df, on="region_code", how="outer")
    else:
        lau_df = _df

In [ ]:
lau_df

In [ ]:
industry_vars = ["number_of_iron_and_steel_industries",
      "number_of_cement_industries",
      "number_of_refineries",
      "number_of_paper_and_printing_industries",
      "number_of_chemical_industries",
      "number_of_glass_industries",
      "number_of_non_ferrous_metals_industries",
      "number_of_non_metallic_minerals_industries",]

air_pollution_vars = ['average_air_pollution_due_to_pm25',
       'average_air_pollution_due_to_no2',
       'average_air_pollution_due_to_o3',
       'average_air_pollution_due_to_pm10',]

In [ ]:
lau_df["number_of_industries"] = lau_df[industry_vars].sum(axis=1)
lau_df.drop(columns=industry_vars, inplace=True)

lau_df["average_air_pollution"] = lau_df[air_pollution_vars].mean(axis=1)
lau_df.drop(columns=air_pollution_vars, inplace=True)

In [ ]:
# Data sources, including Corine Land Cover, OpenStreetMap, and SEEnergies, provide spatial data either as raster files or at 
# specific X-Y coordinates. This spatial information is overlain with LAU regions and aggregated at the LAU level to create 
# regional datasets. If no data points overlap with a given LAU region, the value is set to zero
lau_df = lau_df.fillna(0)

In [ ]:
vars = list(lau_df.columns)
vars.pop(0)

In [ ]:
nuts3_df = None

for var_name in vars:
    _df = lau_df[["region_code", var_name]].copy()

    #convert LAU to NUTS3 regions
    _df["region_code"] = _df["region_code"].str.split("_").str[0]

    # aggregate per NUTS3 region 
    

    if var_name == "average_air_pollution":
        _df = _df.groupby("region_code").mean().reset_index()
    else:
        _df = _df.groupby("region_code").sum().reset_index()
    
    if nuts3_df is not None:
        nuts3_df = pd.merge(nuts3_df, _df, on="region_code", how="outer")
    else:
        nuts3_df = _df

In [ ]:
lau_df_de = lau_df[lau_df["region_code"].str.startswith('DE')].copy()
lau_df_es = lau_df[lau_df["region_code"].str.startswith('ES')].copy()

nuts3_df_de = nuts3_df[nuts3_df["region_code"].str.startswith('DE')].copy()
nuts3_df_es = nuts3_df[nuts3_df["region_code"].str.startswith('ES')].copy()

In [ ]:
lau_df_de.drop(columns="region_code",inplace=True)
lau_df_es.drop(columns="region_code",inplace=True)

nuts3_df_de.drop(columns="region_code",inplace=True)
nuts3_df_es.drop(columns="region_code",inplace=True)

In [ ]:
lau_df.drop(columns="region_code",inplace=True)
nuts3_df.drop(columns="region_code",inplace=True)

In [ ]:
fig = plt.figure(figsize=(13, 5))
gs = fig.add_gridspec(1, 3, wspace=0.1, hspace=0.1)

# Germany - LAU --------
ax1 = plt.subplot(gs[:1, :1])

correlations_de = lau_df_de.corr()

sns.heatmap(correlations_de.abs().round(2), ax=ax1, annot=True, cmap="Blues", cbar=False, vmin=0, vmax=1)

ax1.set_title("Germany")

# Spain - LAU --------
ax2 = plt.subplot(gs[:1, 1:2])

correlations_es = lau_df_es.corr()

sns.heatmap(correlations_es.abs().round(2), ax=ax2, annot=True, cmap="Blues", cbar=False, vmin=0, vmax=1)
ax2.set_title("Spain")
ax2.tick_params(left=False, labelleft=False)

# Correlation difference  --------
ax3 = plt.subplot(gs[:1, 2:])

correlations_diff = correlations_de - correlations_es

sns.heatmap(correlations_diff.abs().round(2), ax=ax3, annot=True, cmap="Reds", cbar=False, vmin=0, vmax=1)
ax3.set_title("Correlations difference")
ax3.tick_params(left=False, labelleft=False)

# First colorbar (for Germany and Spain)
norm1 = plt.Normalize(vmin=0, vmax=1)
cmap1 = "Blues"
sm1 = ScalarMappable(cmap=cmap1, norm=norm1)
sm1.set_array([])

# Second colorbar (for Correlation difference)
norm2 = plt.Normalize(vmin=0, vmax=1)
cmap2 = "Reds"
sm2 = ScalarMappable(cmap=cmap2, norm=norm2)
sm2.set_array([])

# Position the first colorbar between ax1 and ax2
pos1 = ax1.get_position()
pos2 = ax2.get_position()
cbar_x0 = pos1.x0
cbar_width = pos2.x1 - pos1.x0
cbar_y0 = pos1.y0 - 0.4  # Adjust as needed
cbar_height = 0.02        # Adjust as needed
cbar_ax1 = fig.add_axes([cbar_x0, cbar_y0, cbar_width, cbar_height])

# Position the second colorbar below ax3
pos3 = ax3.get_position()
cbar_x0_2 = pos3.x0
cbar_width_2 = pos3.width
cbar_y0_2 = pos3.y0 - 0.4 # Adjust as needed
cbar_height_2 = 0.02        # Adjust as needed
cbar_ax2 = fig.add_axes([cbar_x0_2, cbar_y0_2, cbar_width_2, cbar_height_2])

# Draw the colorbars
cbar1 = plt.colorbar(sm1, cax=cbar_ax1, orientation="horizontal")
cbar1.ax.tick_params(labelsize=10)
cbar1.set_label("Correlation coefficient", fontsize=12)

cbar2 = plt.colorbar(sm2, cax=cbar_ax2, orientation="horizontal")
cbar2.ax.tick_params(labelsize=10)
cbar2.set_label("Difference", fontsize=12)

# Set custom ticks and labels for the second colorbar
cbar2.set_ticks([0, 1])
cbar2.set_ticklabels(['LOW', 'HIGH'])

plt.savefig(os.path.join("..", "..", "figures", f"correlation_country_diff.png"), bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13, 5))
gs = fig.add_gridspec(1, 3, wspace=0.1, hspace=0.1)

# LAU --------
ax1 = plt.subplot(gs[:1, :1])

correlations_lau = lau_df.corr()

sns.heatmap(correlations_lau.abs().round(2), ax=ax1, annot=True, cmap="Blues", cbar=False, vmin=0, vmax=1)

ax1.set_title("LAU")

# NUTS3 --------
ax2 = plt.subplot(gs[:1, 1:2])

correlations_nuts3 = nuts3_df.corr()

sns.heatmap(correlations_nuts3.abs().round(2), ax=ax2, annot=True, cmap="Blues", cbar=False, vmin=0, vmax=1)
ax2.set_title("NUTS3")
ax2.tick_params(left=False, labelleft=False)

# Correlation difference  --------
ax3 = plt.subplot(gs[:1, 2:])

correlations_diff = correlations_lau - correlations_nuts3

sns.heatmap(correlations_diff.abs().round(2), ax=ax3, annot=True, cmap="Reds", cbar=False, vmin=0, vmax=1)
ax3.set_title("Correlations difference")
ax3.tick_params(left=False, labelleft=False)

# First colorbar (for Germany and Spain)
norm1 = plt.Normalize(vmin=0, vmax=1)
cmap1 = "Blues"
sm1 = ScalarMappable(cmap=cmap1, norm=norm1)
sm1.set_array([])

# Second colorbar (for Correlation difference)
norm2 = plt.Normalize(vmin=0, vmax=1)
cmap2 = "Reds"
sm2 = ScalarMappable(cmap=cmap2, norm=norm2)
sm2.set_array([])

# Position the first colorbar between ax1 and ax2
pos1 = ax1.get_position()
pos2 = ax2.get_position()
cbar_x0 = pos1.x0
cbar_width = pos2.x1 - pos1.x0
cbar_y0 = pos1.y0 - 0.4  # Adjust as needed
cbar_height = 0.02        # Adjust as needed
cbar_ax1 = fig.add_axes([cbar_x0, cbar_y0, cbar_width, cbar_height])

# Position the second colorbar below ax3
pos3 = ax3.get_position()
cbar_x0_2 = pos3.x0
cbar_width_2 = pos3.width
cbar_y0_2 = pos3.y0 - 0.4 # Adjust as needed
cbar_height_2 = 0.02        # Adjust as needed
cbar_ax2 = fig.add_axes([cbar_x0_2, cbar_y0_2, cbar_width_2, cbar_height_2])

# Draw the colorbars
cbar1 = plt.colorbar(sm1, cax=cbar_ax1, orientation="horizontal")
cbar1.ax.tick_params(labelsize=10)
cbar1.set_label("Correlation coefficient", fontsize=12)

cbar2 = plt.colorbar(sm2, cax=cbar_ax2, orientation="horizontal")
cbar2.ax.tick_params(labelsize=10)
cbar2.set_label("Difference", fontsize=12)

# Set custom ticks and labels for the second colorbar
cbar2.set_ticks([0, 1])
cbar2.set_ticklabels(['LOW', 'HIGH'])

plt.savefig(os.path.join("..", "..", "figures", f"correlation_spatially_diff.png"), bbox_inches='tight')